In [ ]:
import julia
julia.install()



Precompiling PyCall...
Precompiling PyCall... DONE
PyCall is installed and built successfully.


In [ ]:
import matplotlib.pyplot as plt
from julia import Main

In [ ]:
import Pkg
Pkg.add("QuantumOptics")


   Resolving package versions...
     Project No packages added to or removed from `C:\Users\Vedant\.julia\environments\v1.12\Project.toml`
    Manifest No packages added to or removed from `C:\Users\Vedant\.julia\environments\v1.12\Manifest.toml`


In [ ]:
print("Loading Julia engine...")
# Load Jaynes-Cummings-Hubbard Model functions
Main.include("jch.jl")

Loading Julia engine...

solve_jch (generic function with 1 method)

In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
from qutip import *
from julia import Main


def run_qutip_benchmark(N_list):
    results = []
    w, g, J = 1.0, 0.05, 0.05
    tlist = np.linspace(0, 50, 100)
    
    print("--- Starting QuTiP Benchmark ---")
    for N in N_list:
        # Build System
        a = tensor(destroy(N), qeye(2))
        sm = tensor(qeye(N), sigmam())
        sz = tensor(qeye(N), sigmaz())
        
        # Expand to 2 sites
        a1 = tensor(a, qeye(N), qeye(2))
        sm1 = tensor(sm, qeye(N), qeye(2))
        sz1 = tensor(sz, qeye(N), qeye(2))
        
        a2 = tensor(qeye(N), qeye(2), a)
        sm2 = tensor(qeye(N), qeye(2), sm)
        sz2 = tensor(qeye(N), qeye(2), sz)
        
        # Hamiltonian
        H = w*a1.dag()*a1 + 0.5*w*sz1 + g*(a1.dag()*sm1 + a1*sm1.dag()) + \
            w*a2.dag()*a2 + 0.5*w*sz2 + g*(a2.dag()*sm2 + a2*sm2.dag()) - \
            J*(a1.dag()*a2 + a1*a2.dag())
            
        psi0 = tensor(basis(N,0), basis(2,0), basis(N,0), basis(2,1))
        
        # Measure Time
        start = time.time()
        # Pass empty list [] for e_ops to measure pure solver speed
        mesolve(H, psi0, tlist, [], []) 
        elapsed = time.time() - start
        
        results.append(elapsed)
        print(f"QuTiP N={N}: {elapsed:.4f} s")
    return results

# --- 2. Define Julia Solver (via Bridge) ---
julia_code = """
using QuantumOptics
function solve_jch_pure(N)
    # Setup
    b_cav = FockBasis(N)
    b_atom = SpinBasis(1//2)
    b_site = b_cav ⊗ b_atom
    
    a = destroy(b_cav) ⊗ one(b_atom)
    sm = one(b_cav) ⊗ sigmam(b_atom)
    sz = one(b_cav) ⊗ sigmaz(b_atom)
    
    a1 = a ⊗ one(b_site)
    sm1 = sm ⊗ one(b_site)
    sz1 = sz ⊗ one(b_site)
    a2 = one(b_site) ⊗ a
    sm2 = sm ⊗ one(b_site)
    sz2 = sz ⊗ one(b_site)
    
    H = 1.0*a1'*a1 + 0.5*sz1 + 0.05*(a1'*sm1 + a1*sm1') +
        1.0*a2'*a2 + 0.5*sz2 + 0.05*(a2'*sm2 + a2*sm2') - 
        0.05*(a1'*a2 + a1*a2')
        
    psi0 = fockstate(b_cav, 0) ⊗ spinup(b_atom) ⊗ fockstate(b_cav, 0) ⊗ spindown(b_atom)
    T = [0:0.5:50;]
    
    # Run
    timeevolution.schroedinger(T, psi0, H)
end
"""
Main.eval(julia_code)

def run_julia_benchmark(N_list):
    results = []
    print("--- Starting Julia Benchmark ---")
    
    # Warmup run (to compile JIT)
    print("Warming up Julia...")
    Main.solve_jch_pure(2)
    
    for N in N_list:
        start = time.time()
        Main.solve_jch_pure(N)
        elapsed = time.time() - start
        results.append(elapsed)
        print(f"Julia N={N}: {elapsed:.4f} s")
    return results

# --- 3. Run Comparison ---
# We use N values that differ enough to show scaling
N_values = [2, 4, 6, 8, 10, 12, 20, 30, 40, 50] 

qutip_times = run_qutip_benchmark(N_values)
julia_times = run_julia_benchmark(N_values)

# --- 4. Plot Results ---
plt.figure(figsize=(10, 6))
plt.plot(N_values, qutip_times, 'o-', label="Python (QuTiP)", color="blue", linewidth=2)
plt.plot(N_values, julia_times, 's-', label="Julia (QuantumOptics)", color="red", linewidth=2)

plt.xlabel("Cavity Cutoff (N)")
plt.ylabel("Execution Time (seconds)")
plt.title("Scaling Benchmark: QuTiP vs. Julia")
plt.legend()
plt.grid(True) 
plt.show()

JuliaError: Exception 'ArgumentError: Package QuantumOptics not found in current path.
- Run `import Pkg; Pkg.add("QuantumOptics")` to install the QuantumOptics package.' occurred while calling julia code:

using QuantumOptics
function solve_jch_pure(N)
    # Setup
    b_cav = FockBasis(N)
    b_atom = SpinBasis(1//2)
    b_site = b_cav ⊗ b_atom
    
    a = destroy(b_cav) ⊗ one(b_atom)
    sm = one(b_cav) ⊗ sigmam(b_atom)
    sz = one(b_cav) ⊗ sigmaz(b_atom)
    
    a1 = a ⊗ one(b_site)
    sm1 = sm ⊗ one(b_site)
    sz1 = sz ⊗ one(b_site)
    a2 = one(b_site) ⊗ a
    sm2 = sm ⊗ one(b_site)
    sz2 = sz ⊗ one(b_site)
    
    H = 1.0*a1'*a1 + 0.5*sz1 + 0.05*(a1'*sm1 + a1*sm1') +
        1.0*a2'*a2 + 0.5*sz2 + 0.05*(a2'*sm2 + a2*sm2') - 
        0.05*(a1'*a2 + a1*a2')
        
    psi0 = fockstate(b_cav, 0) ⊗ spinup(b_atom) ⊗ fockstate(b_cav, 0) ⊗ spindown(b_atom)
    T = [0:0.5:50;]
    
    # Run
    timeevolution.schroedinger(T, psi0, H)
end
